In [ ]:
# GAE

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges, negative_sampling
import pandas as pd
import uuid
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, average_precision_score

from ast import literal_eval
import numpy as np

from torch_geometric.nn import Node2Vec

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Load datasets
job_descriptions = pd.read_csv('./data/processed/job_descriptions_processed-a01.csv')
resumes = pd.read_csv('./data/processed/general-resume-dataset-processed-v1-a01.csv', converters={'skills': literal_eval})

# Shuffle job_descriptions and select the first N rows
job_descriptions = job_descriptions.sample(frac=1, random_state=random_seed).head(25000)

# Convert 'skills' column to list
job_descriptions['skills'] = job_descriptions['skills'].apply(literal_eval)

In [ ]:
# Replace None values in job_title and category with a default value before encoding
job_descriptions['job_title'].fillna('unknown', inplace=True)
resumes['job_title'].fillna('unknown', inplace=True)
resumes['category'].fillna('unknown', inplace=True)

# Ensure job_ids and candidate_ids are correctly assigned
job_descriptions['job_id'] = range(1, len(job_descriptions) + 1)
resumes['candidate_id'] = range(1, len(resumes) + 1)

# Add 'unknown' to the list of all titles and categories to handle unseen labels
all_titles = job_descriptions['job_title'].tolist() + resumes['job_title'].tolist()
all_titles.append('unknown')
all_categories = resumes['category'].tolist()
all_categories.append('unknown')

In [ ]:
# Fit the label encoders
le_job_title = LabelEncoder()
le_category = LabelEncoder()
le_job_title.fit(all_titles)
le_category.fit(all_categories)

# Transform the columns
job_descriptions['job_title'] = le_job_title.transform(job_descriptions['job_title'])
resumes['job_title'] = le_job_title.transform(resumes['job_title'])
resumes['category'] = le_category.transform(resumes['category'])

# Encode skills
all_skills = set(skill for skills in job_descriptions['skills'].tolist() + resumes['skills'].tolist() for skill in skills)
le_skills = {skill: i for i, skill in enumerate(all_skills)}

In [ ]:
edge_index = torch.load('./data/features/edge_index-25k-a01.pt')
edge_weight = torch.load('./data/features/edge_weight-25k-a01.pt')
nodes_length = 0
with open('./data/features/nodes_len-a01.txt', 'r') as file:
    nodes_length = np.int64(file.read())

In [ ]:
x = torch.load('./data/features/x-25k-a01.pt')

In [ ]:
data = Data(x=x, edge_index=edge_index, edge_weight=edge_weight)
original_edge_index = data.edge_index.clone()
data = train_test_split_edges(data)

In [ ]:
# Create a dictionary to map edge indices to their weights
edge_weight_dict = {tuple(edge_index[:, i].tolist()): edge_weight[i].item() for i in range(edge_index.size(1))}

def get_edge_weights(edge_index, edge_weight_dict):
    weights = []
    for i in range(edge_index.size(1)):
        edge = tuple(edge_index[:, i].tolist())
        weight = edge_weight_dict.get(edge, 0)  # Default to 0 if edge not found
        weights.append(weight)
    return torch.tensor(weights, dtype=torch.float)

In [ ]:
train_edge_weights = get_edge_weights(data.train_pos_edge_index, edge_weight_dict)
test_edge_weights = get_edge_weights(data.test_pos_edge_index, edge_weight_dict)
val_edge_weights = get_edge_weights(data.val_pos_edge_index, edge_weight_dict)

In [ ]:
data.train_pos_edge_weight = train_edge_weights
data.test_pos_edge_weight = test_edge_weights
data.val_pos_edge_weight = val_edge_weights

# Manually create negative edges for training
neg_edge_index_train = negative_sampling(
    edge_index=data.train_pos_edge_index,
    num_nodes=data.num_nodes,
    num_neg_samples=data.train_pos_edge_index.size(1),
)
data.train_neg_edge_index = neg_edge_index_train

# Assign zero weights to negative edges for training
neg_train_edge_weights = torch.zeros(neg_edge_index_train.size(1), dtype=torch.float)

# Manually create negative edges for testing
neg_edge_index_test = negative_sampling(
    edge_index=data.test_pos_edge_index,
    num_nodes=data.num_nodes,
    num_neg_samples=data.test_pos_edge_index.size(1),
)
data.test_neg_edge_index = neg_edge_index_test

# Assign zero weights to negative edges for testing
neg_test_edge_weights = torch.zeros(neg_edge_index_test.size(1), dtype=torch.float)

# Combine positive and negative edge weights for training
data.train_neg_edge_weight = neg_train_edge_weights

# Combine positive and negative edge weights for testing
data.test_neg_edge_weight = neg_test_edge_weights

# Ensure edge_index tensors are of integer type
data.train_pos_edge_index = data.train_pos_edge_index.long()
data.test_pos_edge_index = data.test_pos_edge_index.long()
data.train_neg_edge_index = data.train_neg_edge_index.long()
data.test_neg_edge_index = data.test_neg_edge_index.long()

In [ ]:
# Define number of nodes
num_nodes = data.num_nodes

# Node2Vec parameters
embedding_dim = 64
walk_length = 20
context_size = 10
walks_per_node = 10
batch_size = 128
lr = 0.01
num_epochs = 21

In [ ]:
# Initialize Node2Vec
node2vec = Node2Vec(
    edge_index=original_edge_index,
    embedding_dim=embedding_dim,
    walk_length=walk_length,
    context_size=context_size,
    walks_per_node=walks_per_node,
    num_negative_samples=1,
    p=1,
    q=1,
    sparse=True,
    num_nodes=num_nodes  # Specify number of nodes
)

# Move to the appropriate device (CPU/GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
node2vec = node2vec.to(device)

In [ ]:
# Optimizer for Node2Vec
optimizer = torch.optim.SparseAdam(node2vec.parameters(), lr=lr)

In [ ]:
# Training loop for Node2Vec
def train_node2vec(num_epochs):
    node2vec.train()
    for epoch in range(num_epochs):
        total_loss = 0
        loader = node2vec.loader(batch_size=batch_size, shuffle=True, num_workers=0)  # Set num_workers=0
        for i, (pos_rw, neg_rw) in enumerate(loader):
            optimizer.zero_grad()
            loss = node2vec.loss(pos_rw.to(device), neg_rw.to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if epoch % 10 == 0:
                print(f'Epoch {epoch + 1}, Iteration {i}, Loss: {total_loss / 10}')
                total_loss = 0

In [ ]:
# Train Node2Vec model
train_node2vec(num_epochs)

In [ ]:
# Extract embeddings
node_embeddings = node2vec.embedding.weight.data.cpu().numpy()

# Update node features with embeddings
data.x = torch.tensor(node_embeddings, dtype=torch.float)

In [ ]:
class GAE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAE, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def encode(self, x, edge_index, edge_weight):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        return self.conv2(x, edge_index, edge_weight)

    def decode(self, z, pos_edge_index, neg_edge_index):
        pos_pred = (z[pos_edge_index[0].long()] * z[pos_edge_index[1].long()]).sum(dim=1)
        neg_pred = (z[neg_edge_index[0].long()] * z[neg_edge_index[1].long()]).sum(dim=1)
        return pos_pred, neg_pred

    def forward(self, data):
        z = self.encode(data.x, data.train_pos_edge_index, data.train_pos_edge_weight)
        return z

In [ ]:
# Initialize and train GAE model as before
model = GAE(data.num_node_features, 32)  # Adjust dimensions as needed
gae_optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
def train(data):
    model.train()
    gae_optimizer.zero_grad()
    z = model.encode(data.x, data.train_pos_edge_index, data.train_pos_edge_weight)  # Pass train_pos_edge_weight
    pos_pred, neg_pred = model.decode(z, data.train_pos_edge_index, data.train_neg_edge_index)
    pos_loss = loss_fn(pos_pred, torch.ones_like(pos_pred))
    neg_loss = loss_fn(neg_pred, torch.zeros_like(neg_pred))
    loss = pos_loss + neg_loss
    if torch.isnan(loss) or torch.isinf(loss):
        print("Warning: NaN or Inf loss detected")
        return float('inf')
    loss.backward()
    gae_optimizer.step()
    return loss.item()

In [ ]:
for epoch in range(30):
    loss = train(data)
    if loss == float('inf'):
        break
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss}')

In [ ]:
def precision_at_k(y_true, y_pred, k):
    idx = np.argsort(y_pred)[::-1][:k]
    y_pred_binary = np.zeros_like(y_pred)
    y_pred_binary[idx] = 1
    tp = np.sum(y_true * y_pred_binary)
    precision = tp / k
    return precision

def recall_at_k(y_true, y_pred, k):
    idx = np.argsort(y_pred)[::-1][:k]
    y_pred_binary = np.zeros_like(y_pred)
    y_pred_binary[idx] = 1
    tp = np.sum(y_true * y_pred_binary)
    recall = tp / np.sum(y_true)
    return recall

def average_precision(y_true, y_pred):
    idx = np.argsort(y_pred)[::-1]
    y_true_sorted = y_true[idx]
    tp = np.cumsum(y_true_sorted)
    precision = tp / (np.arange(len(y_true_sorted)) + 1)
    avg_precision = np.sum(precision * y_true_sorted) / np.sum(y_true_sorted)
    return avg_precision

def mean_average_precision(y_true, y_pred):
    return np.mean([average_precision(y_t, y_p) for y_t, y_p in zip(y_true, y_pred)])

def dcg_score(y_true, y_pred, k):
    order = np.argsort(y_pred)[::-1]
    y_true_sorted = np.take(y_true, order[:k])
    gain = 2 ** y_true_sorted - 1
    discounts = np.log2(np.arange(len(y_true_sorted)) + 2)
    return np.sum(gain / discounts)

def ndcg_score(y_true, y_pred, k):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_pred, k)
    return actual / best

In [ ]:
def evaluate_model(data, model, k):
    model.eval()
    with torch.no_grad():
        z = model.encode(data.x, data.val_pos_edge_index, data.val_pos_edge_weight)
        pos_pred = torch.sigmoid((z[data.val_pos_edge_index[0].long()] * z[data.val_pos_edge_index[1].long()]).sum(dim=1)).cpu().numpy()
        neg_pred = torch.sigmoid((z[data.val_neg_edge_index[0].long()] * z[data.val_neg_edge_index[1].long()]).sum(dim=1)).cpu().numpy()

    y_true = np.concatenate([np.ones(pos_pred.shape[0]), np.zeros(neg_pred.shape[0])])
    y_pred = np.concatenate([pos_pred, neg_pred])

    auc_roc = roc_auc_score(y_true, y_pred)
    ap = average_precision_score(y_true, y_pred)

    precision = precision_at_k(y_true, y_pred, k)
    recall = recall_at_k(y_true, y_pred, k)
    map_score = mean_average_precision([y_true], [y_pred])
    ndcg = ndcg_score(y_true, y_pred, k)

    return auc_roc, ap, precision, recall, map_score, ndcg

In [ ]:
k=10

In [ ]:
# Example usage
auc_roc, ap, k_prec, recall, map_score, ndcg = evaluate_model(data, model, k)
print(f"AUC-ROC: {auc_roc:.4f}, AP: {ap:.4f}")
print(f"Precision@{k}: {k_prec:.4f}")
print(f"Recall@{k}: {recall:.4f}")
print(f"MAP: {map_score:.4f}")
print(f"NDCG@{k}: {ndcg:.4f}")

In [ ]:
def predict_best_candidates(job_descriptions, resumes, z, k=1):
    job_ids = job_descriptions['job_id'].values
    candidate_ids = resumes['candidate_id'].values + len(job_descriptions)

    job_indices = job_ids - 1
    candidate_indices = candidate_ids - 1

    job_embeddings = z[job_indices]
    candidate_embeddings = z[candidate_indices]

    # Calculate scores using matrix multiplication
    scores = torch.sigmoid(torch.matmul(job_embeddings, candidate_embeddings.T)).cpu().numpy()

    predictions = []
    for i, job_id in enumerate(job_ids):
        best_match_indices = scores[i].argsort()[::-1][:k]
        for idx in best_match_indices:
            candidate_id = resumes.iloc[idx]['candidate_id']
            candidate_job_title = resumes.iloc[idx]['job_title']
            category = resumes.iloc[idx]['category']
            skills = resumes.iloc[idx]['skills']
            job_title = job_descriptions.iloc[i]['job_title']
            job_skills = job_descriptions.iloc[i]['skills']
            score = scores[i][idx]

            match_percentage = score * 100  # Assuming the score is between 0 and 1
            predictions.append({
                "Job ID": job_id,
                "Job Title": le_job_title.inverse_transform([job_title])[0],
                "Candidate ID": candidate_id,
                "Candidate Job Title": le_job_title.inverse_transform([candidate_job_title])[0],
                "Candidate Category": le_category.inverse_transform([category])[0],
                "Match Percentage": match_percentage,
                "Mutual Skills": set(job_skills).intersection(set(skills)),
                "Job Skills": job_skills,
                "Candidate Skills": skills
            })

    predictions_df = pd.DataFrame(predictions)
    return predictions_df

In [ ]:
# Example usage
with torch.no_grad():
    z = model.encode(data.x, data.test_pos_edge_index, data.test_pos_edge_weight)

In [ ]:
jobs_to_predict = job_descriptions[job_descriptions['job_title'].isin(resumes['job_title'])].sample(frac=1, random_state=random_seed)

In [ ]:
predictions_df = predict_best_candidates(jobs_to_predict, resumes, z)
predictions_df = predictions_df[predictions_df['Mutual Skills'].map(len) != 0]

In [ ]:
predictions_df.head(75000)

In [86]:
torch.save(model.state_dict(), f"./models/gea-recommendation-system-v4-a01-1.pth")